In [5]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from nmf_methods.nmf_son.base import nmf_son as nmf_son_base
from nmf_methods.nmf_son.new import nmf_son_constrained_H
from nmf_methods.nmf_son.new import nmf_son_new
from nmf_methods.nmf_son.utils import save_results, load_results

np.random.seed(42)
np.set_printoptions(precision=3)

## Toy Example

In [6]:
def create_toy_ex(n):
    W = np.random.rand(2, 3)
    H = np.ones((3, n))
    thres = 0.88
    id = np.argwhere(np.sum(H >= thres, axis=0))
    while id.any():
        id = np.argwhere(np.sum(H >= thres, axis=0))
        H[:, id.flatten()] = np.random.dirichlet((0.33, 0.33, 0.33), len(id)).T

    M = W @ H
    return M, W, H


def plot_scores(fscores, gscores, lambda_vals, plot_title=None, filename=None):
    fscores = fscores[1:]
    gscores = gscores[1:]
    lambda_vals = lambda_vals[1:]
    total_score = fscores + lambda_vals * gscores
    fig, axs = plt.subplots(1, 2, figsize=(20, 5))
    if plot_title:
        fig.suptitle(plot_title, fontsize=25)

    axs[0].set_yscale('log')
    axs[0].plot(total_score, color='black', linewidth=3, label='$F(W, H)$')
    axs[0].plot(fscores, color='cyan', linewidth=1.5, label='$f(W, H)$')
    axs[0].plot(gscores, color='yellow', linewidth=1.5, label='$g(W)$')
    axs[0].set_xlabel('Iterations')
    axs[0].legend()

    fscores -= min(fscores)
    gscores -= min(gscores)
    total_score -= min(total_score)

    axs[1].set_yscale('log')
    axs[1].plot(total_score, color='black', linewidth=3, label='$F(W, H) - min(F(W, H))$')
    axs[1].plot(fscores, color='cyan', linewidth=1.5, label='$f(W, H) - min(f(W, H))$')
    axs[1].plot(gscores, color='yellow', linewidth=1.5, label='$g(W) - min(g(W))$')
    axs[1].set_xlabel('Iterations')
    axs[1].legend()

    if filename:
        fig.savefig(filename)


def plot_mats(ax, M, W, W_true):
    symbols = ['o', 'x', 'v', 's', '.']
    for i in range(len(symbols)):
        ax.plot(W[0, i], W[1, i],f'r{symbols[i]}', markersize=5, linewidth=2)
    ax.plot(M[0, :], M[1, :],'k.')

    for j in range(W_true.shape[1]):
        ax.plot(W_true[0, j], W_true[1, j],'bx', markersize=5, linewidth=2)


def plot_seperate_H(H, img_size, figsize, fontsize, normalize_row=False, split=False, filename=None):
    rank = H.shape[0]
    if normalize_row:
        H /= np.linalg.norm(H, axis=1, keepdims=True)
    H3d = H.reshape(-1, img_size[0], img_size[1], order='F')
    if split:
        half_rank = int(rank / 2)
        fig, axs = plt.subplots(2, half_rank, figsize=figsize)
        i, j, cnt = 0, 0, 0
        while cnt < rank:
            img = axs[i, j].imshow(H3d[cnt, :, :], cmap='gray')
            axs[i, j].set_title(f'$h^{cnt + 1}$', fontsize=fontsize)
            axs[i, j].axis('off')
            divider = make_axes_locatable(axs[i, j])
            cax = divider.append_axes('right', size='5%', pad=0.1)
            fig.colorbar(img, cax=cax, orientation='vertical')



            j += 1
            if cnt + 1 == half_rank:
                i = 1
                j = 0
            cnt += 1
    else:
        fig, axs = plt.subplots(1, rank, figsize=figsize)
        cnt = 0
        while cnt < rank:
            img = axs[cnt].imshow(H3d[cnt, :, :], cmap='gray')
            axs[cnt].set_title(f'$h^{cnt + 1}$', fontsize=fontsize)
            axs[cnt].axis('off')
            divider = make_axes_locatable(axs[cnt])
            cax = divider.append_axes('right', size='5%', pad=0.1)
            fig.colorbar(img, cax=cax, orientation='vertical')

            cnt += 1
    # plt.tight_layout()
    if filename:
        plt.savefig(filename)


def plot_combined_H(H, img_size, figsize, normalize_row=False, split=False, filename=None):
    if normalize_row:
        H /= np.linalg.norm(H, axis=1, keepdims=True)
    H3d = H.reshape(-1, img_size[0], img_size[1], order='F')
    if split:
        half_rank = int(H.shape[0] / 2)
        large_mat = np.vstack([np.hstack(H3d[: half_rank, :, :]), np.hstack(H3d[half_rank: , :, :])])
    else:
        large_mat = np.hstack(H3d)
    plt.figure(figsize=figsize)
    ax = plt.axes()
    im = plt.imshow(large_mat, cmap='gray')

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    plt.colorbar(im, cax=cax)

    if filename:
        plt.savefig(filename)


def plot_W_mats(W, figsize, fontsize, split=False, filename=None, scale_y=False, log_scale=False, plot_title=None):
    rank = W.shape[1]
    wmin = np.min(W)
    wmax = np.max(W)

    if split:
        half_rank = int(rank / 2)
        fig, axs = plt.subplots(2, half_rank, figsize=figsize)
        i, j, cnt = 0, 0, 0
        while cnt < rank:
            axs[i, j].plot(W[:, cnt], linewidth=3)
            if scale_y:
                axs[i, j].set_ylim([min(0, wmin), wmax])
            if log_scale:
                axs[i, j].set_yscale('log')
            axs[i, j].set_title(f'$w_{cnt + 1}$', fontsize=fontsize)
            axs[i, j].set_xlabel(f'Bands')
            axs[i, j].set_ylabel(f'Reflectance')
            j += 1
            if cnt + 1 == half_rank:
                i = 1
                j = 0
            cnt += 1
    else:
        fig, axs = plt.subplots(1, rank, figsize=figsize)
        cnt = 0
        while cnt < rank:
            axs[cnt].plot(W[:, cnt], linewidth=3)
            if scale_y:
                axs[cnt].set_ylim([min(0, wmin), wmax])
            if log_scale:
                axs[cnt].set_yscale('log')
            axs[cnt].set_title(f'$w_{cnt + 1}$', fontsize=fontsize)
            axs[cnt].set_xlabel(f'Bands')
            axs[cnt].set_ylabel(f'Reflectance')

            cnt += 1
    plt.tight_layout()

    if plot_title:
        fig.suptitle(plot_title, fontsize=25)
    if filename:
        fig.savefig(filename)

def flatten_irr_2d(arr_2d):
    flat_arr = list()
    for arr in arr_2d:
        for elem in arr:
            if elem not in flat_arr:
                flat_arr.append(elem)

    return flat_arr

In [7]:
max_iters = 10000

In [8]:
EARLY_STOP = True
VERBOSE = False
SCALE_REG = True

In [ ]:
# toy
# M, W_true, H = create_toy_ex(30)
# with open('../experimental/datasets/toy_n30.npz', 'wb') as fout:
#     np.savez_compressed(fout, M=M, W=W_true, H=H)


data = np.load('../experimental/datasets/toy_n30.npz')
M = data['M']
W_true = data['W']
H = data['H']

m, n = M.shape
r = 5

lambda_vals = [1e-07, 1e-06, 1e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 50, 100, 500, 1000]
# lambda_vals = [10, 10.25, 10.5, 10.75, 11, 11.25, 11.5, 11.75, 12, 12.25, 12.5, 12.75, 13, 13.25, 13.5, 13.75, 14]
# lambda_vals = [10.25, 10.5, 10.75, 11, 11.25, 11.5, 11.75, 12.25, 12.5, 12.75, 13, 13.25, 13.5, 13.75]


ini_filepath = f'../experimental/saved_models/toy/r{r}_ini.npz'
save_filepath = '../experimental/saved_models/toy/r{}_l{}_mit{}.npz'

data = np.load(ini_filepath)
ini_W = data['ini_W']
ini_H = data['ini_H']

for _lam in lambda_vals:
    # W, H, fscores, gscores, lvals = nmf_son_new(M, ini_W.copy(), ini_H.copy(), _lambda=_lam, itermax=max_iters, early_stop=EARLY_STOP, verbose=VERBOSE, scale_reg=SCALE_REG)
    # save_results(save_filepath.format(r, _lam, max_iters), W, H, fscores, gscores, lvals)
    # print(_lam)

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    W, H, fscores, gscores, lvals = load_results(save_filepath.format(r, _lam, max_iters))
    axs[0].set_title(f'$\lambda = {_lam}$')
    plot_mats(axs[0], M, W, W_true)
    # fscores -= min(fscores)
    # gscores -= min(gscores)
    total_score = fscores + lvals * gscores
    # total_score -= min(total_score)
    axs[1].set_yscale('log')
    axs[1].plot(total_score, color='black', linewidth=3, label='$F(W, H) - min(F(W, H))$')
    axs[1].plot(fscores, color='cyan', linewidth=1.5, label='$f(W, H) - min(f(W, H))$')
    axs[1].plot(gscores, color='yellow', linewidth=1.5, label='$g(W) - min(g(W))$')
    axs[1].set_xlabel('Iterations')
    axs[1].legend()

In [ ]:
np.linspace(10, 14, 17)

In [ ]:
# jasper
M = np.load('../experimental/datasets/jasper_full.npz')['X']
m, n = M.shape
r = 8

lambda_vals = [1e-07, 1e-06, 1e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1, 2, 5, 8, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 70, 80, 100, 500, 1000]

ini_filepath = f'../experimental/saved_models/jasper/r{r}_ini.npz'
save_filepath = '../experimental/saved_models/jasper/r{}_l{}_mit{}.npz'


data = np.load(ini_filepath)
ini_W = data['ini_W']
ini_H = data['ini_H']

with open(ini_filepath, 'wb') as fout:
    np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)

for _lam in lambda_vals[:30]:
    # W, H, fscores, gscores, lvals = nmf_son_new(M, ini_W.copy(), ini_H.copy(), _lambda=_lam, itermax=max_iters, early_stop=EARLY_STOP, verbose=VERBOSE, scale_reg=SCALE_REG)
    # save_results(save_filepath.format(r, _lam, max_iters), W, H, fscores, gscores, lvals)

    W, H, fscores, gscores, lvals = load_results(save_filepath.format(r, _lam, max_iters))
    plot_scores(fscores, gscores, lvals, plot_title=_lam)
    plot_W_mats(W, figsize=(28, 6), fontsize=25, scale_y=True)
    plot_seperate_H(H, (100, 100), figsize=(20, 5), fontsize=15, normalize_row=True, split=True)


In [ ]:
# urban
M = np.load('../experimental/datasets/urban_full.npz')['X']
m, n = M.shape
r = 10

lambda_vals = [1e-7, 1e-6, 1e-5, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000]

ini_filepath = f'../experimental/saved_models/urban/r{r}_ini.npz'
save_filepath = '../experimental/saved_models/urban/r{}_l{}_it{}.npz'

data = np.load(ini_filepath)
ini_W = data['ini_W']
ini_H = data['ini_H']

with open(ini_filepath, 'wb') as fout:
    np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)

for _lam in lambda_vals:
    W, H, fscores, gscores, lvals = nmf_son_new(M, ini_W.copy(), ini_H.copy(), _lambda=_lam, itermax=max_iters, early_stop=EARLY_STOP, verbose=VERBOSE, scale_reg=SCALE_REG)
    save_results(save_filepath.format(r, _lam, max_iters), W, H, fscores, gscores, lvals)
    print(_lam)

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython import display

Figure = plt.figure()

# creating a plot
lines_plotted = plt.plot([])
line_plotted = lines_plotted[0]

plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)

def anim_func(frame):

	# setting y according to frame
	# number and + x. It's logic
	y = np.cos(x+2*np.pi*frame/100)

	# line is set with new values of x and y
	line_plotted.set_data((x, y))

anim_created = FuncAnimation(Figure, AnimationFunction, frames=100, interval=25)

video = anim_created.to_html5_video()
html = display.HTML(video)
display.display(html)

# good practice to close the plt object.
plt.close()